# SOM Coding
## Group Topic B Dendrogramm
- Peter Blohm (11905150)
- Martin Braunsperger (11909911)
- Adrian Gawor (11905152)

## Github
https://github.com/Pietreus/sos-2022w-exercise3

In [ ]:
import numpy as np
import networkx as nx
from networkx.algorithms import tree
import pandas as pdcoding
import gzip

In [ ]:
#SOMToolbox Parser
from SOMToolBox_Parse import SOMToolBox_Parse
idata = SOMToolBox_Parse("datasets/10clusters/10clusters.vec").read_weight_file()
weights = SOMToolBox_Parse("datasets/iris/iris.wgt.gz").read_weight_file()
weights

In [ ]:
#HitHistogram
def HitHist(_m, _n, _weights, _idata):
    hist = np.zeros(_m * _n)
    for vector in _idata: 
        position =np.argmin(np.sqrt(np.sum(np.power(_weights - vector, 2), axis=1)))
        hist[position] += 1

    return hist.reshape(_m, _n)

#U-Matrix - implementation
def UMatrix(_m, _n, _weights, _dim):
    U = _weights.reshape(_m, _n, _dim)
    U = np.insert(U, np.arange(1, _n), values=0, axis=1)
    U = np.insert(U, np.arange(1, _m), values=0, axis=0)
    #calculate interpolation
    for i in range(U.shape[0]): 
        if i%2==0:
            for j in range(1,U.shape[1],2):
                U[i,j][0] = np.linalg.norm(U[i,j-1] - U[i,j+1], axis=-1)
        else:
            for j in range(U.shape[1]):
                if j%2==0: 
                    U[i,j][0] = np.linalg.norm(U[i-1,j] - U[i+1,j], axis=-1)
                else:      
                    U[i,j][0] = (np.linalg.norm(U[i-1,j-1] - U[i+1,j+1], axis=-1) + np.linalg.norm(U[i+1,j-1] - U[i-1,j+1], axis=-1))/(2*np.sqrt(2))

    U = np.sum(U, axis=2) #move from Vector to Scalar

    for i in range(0, U.shape[0], 2): #count new values
        for j in range(0, U.shape[1], 2):
            region = []
            if j>0: region.append(U[i][j-1]) #check left border
            if i>0: region.append(U[i-1][j]) #check bottom
            if j<U.shape[1]-1: region.append(U[i][j+1]) #check right border
            if i<U.shape[0]-1: region.append(U[i+1][j]) #check upper border

            U[i,j] = np.median(region)

    return U

#SDH - implementation
def SDH(_m, _n, _weights, _idata, factor, approach):
    import heapq

    sdh_m = np.zeros( _m * _n)

    cs=0
    for i in range(factor): cs += factor-i

    for vector in _idata:
        dist = np.sqrt(np.sum(np.power(_weights - vector, 2), axis=1))
        c = heapq.nsmallest(factor, range(len(dist)), key=dist.__getitem__)
        if (approach==0): # normalized
            for j in range(factor):  sdh_m[c[j]] += (factor-j)/cs 
        if (approach==1):# based on distance
            for j in range(factor): sdh_m[c[j]] += 1.0/dist[c[j]] 
        if (approach==2): 
            dmin, dmax = min(dist[c]), max(dist[c])
            for j in range(factor): sdh_m[c[j]] += 1.0 - (dist[c[j]]-dmin)/(dmax-dmin)

    return sdh_m.reshape(_m, _n)

In the following section we define the function used for clustering the weights of the SOM and generating the lines of the dendrogram and minimum spanning tree. First we perform agglomerative clustering by using sklearn's AgglomerativeClustering with the user defined linkage method (defaults to ward). Then we call either mst_line_overlay() to generate the lines of the minimum spanning tree or clustering_line_overlay() to generate the lines of the dendrogram.

In [ ]:
from sklearn.cluster import AgglomerativeClustering


def Clustering(m, n, weights, n_clusters=2, linkage='ward', line_distance_cutoff = 10, max_width = 0.8, inter_cluster = False, intra_cluster = True, line_weighting_method = None):
    """Perform agglomerative clustering/MST on data

    arguments:
        m:  rows of som
        n:  columns of som
        weights:    weight vectors of instances
        n_clusters: amount of clusters to color
        linkage:    linkage method by agglomerative clustering or TODO 'mst'

    Returns:
        (clust_mat,line_dict) where
        clust_mat is an array of cluster labels of shape (m,n)
        line_dict is a dictionary containing line segements and width in the format required by holoviews

    """
    num_samples = len(weights)
    if linkage == "mst":
        lines = mst_line_overlay(m, n, weights, line_weighting_method, max_width)
        clustering = AgglomerativeClustering(n_clusters=1,
                                             compute_distances=False)
        clustering.labels_ = np.ones(m*n)
    else:
        clustering = AgglomerativeClustering(n_clusters=n_clusters,
                                         compute_full_tree=True,
                                         linkage=linkage,
                                         compute_distances=True)
        clustering.fit(weights)
    # clustering.labels_ now contains the labels we want to plot

        lines = clustering_line_overlay(m,
                                        n,
                                        clustering.children_,
                                        clustering.distances_,
                                        num_samples,
                                        n_clusters,
                                        line_distance_cutoff = line_distance_cutoff,
                                        max_width = max_width,
                                        inter_cluster_lines = inter_cluster,
                                        intra_cluster_lines = intra_cluster,
                                        line_weighting_method= line_weighting_method
        )

    return clustering.labels_.reshape(m, n), {'x0':lines.T[0],'y0':lines.T[1],'x1':lines.T[2],'y1':lines.T[3], 'distance': lines.T[4]}


The following code generates the dendrogram based on the clustering information from the agglomerative clustering.

In [ ]:
def clustering_line_overlay(m, n, children, distances, num_samples, n_clusters, inter_cluster_lines, intra_cluster_lines, line_weighting_method, line_distance_cutoff, max_width):
    """defines the lines for the dendrogram line overlay"""

    # full clustering information is contained in clustering.children_
    # row i defines cluster num_samples+i by merging the two clusters in that row

    lines = np.zeros((num_samples - 1,5))

    cluster_positions = np.zeros((2*num_samples - 1,2)) # center of mass for each cluster to plot the line segments from
    cluster_sizes = np.ones((2*num_samples - 1)) # number of units in a cluster to compute center of mass correctly
    for i in range(num_samples):
        cluster_positions[i] = [i % n + .5, m - .5 - i // n] #using modulo operations to get the position

    biggest_cluster = num_samples - 1 - n_clusters
    if inter_cluster_lines:
        max_dist = np.max(distances)
    else:
        max_dist = np.max(distances[:biggest_cluster])


    for index, (child1, child2) in enumerate(children):
        i = index + num_samples #offset for the single unit clusters
        s1 = cluster_sizes[child1]
        s2 = cluster_sizes[child2]
        cluster_sizes[i] = s1+s2
        # calculate center of mass
        cluster_positions[i] = (cluster_positions[child1] * s1 + cluster_positions[child2] * s2)/cluster_sizes[i]


        if (index <= biggest_cluster and intra_cluster_lines) or (index > biggest_cluster and inter_cluster_lines):

            width = 0
            if line_distance_cutoff is None or distances[index]/max_dist <= line_distance_cutoff:
                width = get_line_width(line_weighting_method,distances[index],max_dist,(s1+s2) / num_samples)
            width *= max_width
            lines[index] = np.hstack((cluster_positions[child1],cluster_positions[child2],width))
    return lines


In [ ]:

def index_to_coords(m,n,i):
    return i % n + .5, m - .5 - i // n

def mst_line_overlay(m, n, weights, method, max_width):
    G = nx.Graph()
    for i,v1 in enumerate(weights):
        for j,v2 in enumerate(weights[i+1:]):
            distance = np.linalg.norm(v1-v2)
            G.add_edge(i,j+i+1,weight=distance)

    MST = list(tree.minimum_spanning_edges(G, algorithm="prim", data=True))
    max_weight = max(map(lambda edge_tuple: edge_tuple[2]['weight'],MST))
    lines = []
    for v1, v2, w in MST:
        line = [index_to_coords(m,n,v1),index_to_coords(m,n,v2)]
        line = list(sum(line,())) #flattens to list
        line.append(get_line_width(method,w['weight'],max_weight)*max_width)
        lines.append(line)
        
    return np.array(lines)

def get_line_width(method, distance, max_dist, relative_cluster_size = 1):
    if method is None:
        width = 1
    elif method == "size":
        width = relative_cluster_size ** 0.5
    elif method == "distance":
        width = distance / max_dist
    elif method == "inverse_distance":
        width = (max_dist - distance) / max_dist
    else:
        width = 1
    width = np.clip(width, 0, 1)
    return width

# actual_weights = weights["arr"]
# print(len(actual_weights)-1)
# print(mst_line_overlay(weights['ydim'], weights['xdim'],actual_weights, len(actual_weights)-1))


In [ ]:
# import holoviews as hv
# hv.extension('bokeh')

# hithist = hv.Image(HitHist(weights['ydim'], weights['xdim'], weights['arr'], idata['arr'])).opts(xaxis=None, yaxis=None)
# um = hv.Image(UMatrix(weights['ydim'], weights['xdim'], weights['arr'], 10)).opts(xaxis=None, yaxis=None)
# sdh = hv.Image(SDH(weights['ydim'], weights['xdim'], weights['arr'], idata['arr'], 25, 0)).opts(xaxis=None, yaxis=None)
# hv.Layout([hithist.relabel('HitHist').opts(cmap='kr'),
#            um.relabel('U-Matrix').opts(cmap='jet'), sdh.relabel('SDH').opts(cmap='viridis')])

In [ ]:
import holoviews as hv
hv.extension('bokeh')

def clustering_vis(weights,n_clust,linkage,line_distance_cutoff,max_width, inter_cluster = True, intra_cluster = False, line_weighting_method = "size"):
    clust,lines = Clustering(weights['ydim'], weights['xdim'], weights['arr'], n_clust, linkage, line_distance_cutoff = line_distance_cutoff, max_width = max_width, inter_cluster = inter_cluster, intra_cluster = intra_cluster, line_weighting_method = line_weighting_method)
    img = hv.Image(clust, bounds = (0,0,weights['xdim'], weights['ydim'])).opts(cmap='viridis',width=800,height=800)
    seg = hv.Segments(lines,['x0','y0','x1','y1'],['distance'],).opts( line_width = 5*hv.dim('distance'), color="k")
    return hv.Layout([img*seg])

## Demo Visualisations

### Training the SOMs

In [ ]:
from minisom import MiniSom
from SOMToolBox_Parse import SOMToolBox_Parse

def train_som(path, x, y, iterations = 10000):
    idata = SOMToolBox_Parse(path).read_weight_file()
    dim = idata['vec_dim']
    som = MiniSom(y, x, dim)
    som.pca_weights_init(idata['arr'])
    som.train(idata['arr'], iterations)
    weights = np.transpose(som.get_weights(),(1,0,2)).reshape(-1, som.get_weights().shape[2])
    weight_file = {'xdim':y, 'ydim':x, 'vec_dim':dim, 'arr':weights}
    return idata, weight_file

_, som_10clusters_small_weights = train_som("datasets/10clusters/10clusters.vec", 10, 10)
_, som_10clusters_large_weights = train_som("datasets/10clusters/10clusters.vec", 100, 60, 10000)
_, som_chailink_small_weights = train_som("datasets/chainlink/chainlink.vec", 10, 10)
_, som_chainlink_large_weights = train_som("datasets/chainlink/chainlink.vec", 100, 60)

In [ ]:
clustering_vis(som_chainlink_large_weights,1000,"single",None,1, inter_cluster=False, intra_cluster=True)
#som_chainlink_large_weights['arr']

### Comparison with Java SOM

In [ ]:
som_10clusters_small_weights = SOMToolBox_Parse("datasets/java_som/10clusters_small/10clusters_small.wgt.gz").read_weight_file()
som_10clusters_large_weights = SOMToolBox_Parse("datasets/java_som/10clusters/10clusters.wgt.gz").read_weight_file()
som_chainlink_small_weights = SOMToolBox_Parse("datasets/java_som/chainlink_small/10chainlink_small.wgt.gz").read_weight_file()
som_chainlink_large_weights = SOMToolBox_Parse("datasets/java_som/chainlink/10chainlink.wgt.gz").read_weight_file()

### Parameter Settings

In [ ]:
clustering_vis(som_chainlink_large_weights, 10, "ward", None, 0.8, intra_cluster = True, inter_cluster = False, line_weighting_method="inverse_distance")

In [ ]:
clustering_vis(som_chainlink_small_weights, 10, "mst", None, 1)

In [ ]:
clustering_vis(som_10clusters_large_weights, 10, "ward", None, 1)

In [ ]:
clustering_vis(som_chainlink_small_weights, 7, "single", None, 1, inter_cluster = False, intra_cluster = True)

In [ ]:
clustering_vis(som_chainlink_small_weights, 7, "single", None, 1, inter_cluster = True, intra_cluster = False)

In [ ]:
clustering_vis(som_chainlink_large_weights, 2500, "single", 0, 1)
#more clusters than actual samples (1000)

In [ ]:
clustering_vis(som_chainlink_large_weights, 25, "ward", 2, 2)
#chainlink structure hard to see

In [ ]:
clustering_vis(som_chainlink_large_weights, 1, "mst", 1, 0.8)
#chainlink structure hard to see

### Interactive Demo